# Install requirements

In [ ]:
!pip install transformers torch

# Import necessary libries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Loading PreTrained-Model

In [ ]:
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


# Define the Generative QA Function

In [ ]:
def generative_qa(question,context=None):
    """
    Generate an answer to a question using FLAN-T5
    """

    # If the user provides context, the model is instructed
    # to use that information while answering the question

    if context:
        prompt = f"""
        Use the following context to answer the question.
        Context: {context}

        Question: {question}
        Answer in simple terms:
        """

    # If no context is provided, the model relies on its
    # pre-trained knowledge to generate an answer
    else:
       prompt = f"""
        "You are a helpful assistant.\n"
        "Explain the following question in simple and clear language.\n\n"
        f"Question: {question}\n"
        "Answer:
        """


    # Convert the prompt text into tokens that the model understands
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    # Generate the answer using beam search for better quality responses
    outputs = model.generate(
        **inputs,
        max_new_tokens=180,
        num_beams=5,
        no_repeat_ngram_size=2
    )


    # Convert generated tokens back into human-readable text
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Chatbot

In [ ]:
def chatbot_run():
    print("Generative QA Chatbot")
    print("Type 'exit' to quit\n")

    # Optional context
    use_context = input("Do you want to provide context? (yes/no): ").lower()
    context = ""

    if use_context == "yes":
        print("\nEnter context (press Enter when done):")
        context = input()

    while True:
        question = input("\nYou: ")

        if question.lower() == "exit":
            print("Bot: Goodbye! 👋")
            break

        if context:
            answer = generative_qa(question, context)
        else:
            answer = generative_qa(question)

        print("Bot:", answer)


# Testing

In [ ]:
chatbot_run()

🤖 Generative QA Chatbot
Type 'exit' to quit

Do you want to provide context? (yes/no): yes

Enter context (press Enter when done):
Machine learning is a branch of artificial intelligence (AI) that focuses on developing algorithms that allow computers to learn from data and make predictions. It includes techniques like supervised learning, where labeled data is used to train models, and unsupervised learning, which finds patterns in unlabeled data. Popular ML algorithms include decision trees, support vector machines, and neural networks. Frameworks like TensorFlow and PyTorch are commonly used for building and training ML models

You: what is machine learning
Bot: focuses on developing algorithms that allow computers to learn from data and make predictions

You: tell me about supervied learning
Bot: It includes techniques that allow computers to learn from data and make predictions. It features techniques like supervised learning, where labeled data is used to train models, and unsuper

In [ ]:
chatbot_run()

🤖 Generative QA Chatbot
Type 'exit' to quit

Do you want to provide context? (yes/no): no

You: What is a computer?
Bot: A computer is a piece of hardware.

You: What is the internet?
Bot: The Internet is a network of computers that can be accessed from anywhere in the world.

You: what is data?
Bot: Data refers to information that can be gathered and analyzed.

You: exit
Bot: Goodbye! 👋


In [ ]:
chatbot_run()

Generative QA Chatbot
Type 'exit' to quit

Do you want to provide context? (yes/no): no

You: what is machine learning?
Bot: Machine learning (MLA) is a type of artificial intelligence (AI).


KeyboardInterrupt: Interrupted by user

# Gradio UI

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
def gradio_chatbot(question, context):
    """
    This function is called by Gradio.
    It receives question and optional context from the UI.
    """

    if context.strip() != "":
        answer = generative_qa(question, context)
    else:
        answer = generative_qa(question)

    return answer


In [ ]:
interface = gr.Interface(
    fn=gradio_chatbot,
    inputs=[
        gr.Textbox(
            lines=4,
            label="Context (Optional)",
            placeholder="Enter background information here..."
        ),
        gr.Textbox(
            lines=2,
            label="Question",
            placeholder="Type your question here..."
        )
    ],
    outputs=gr.Textbox(label="Bot Answer"),
    title="Generative QA Chatbot",
    description="Provide optional context to improve answer accuracy."
)


In [ ]:
interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fec0dd842bb56d6a11.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
